<a href="https://colab.research.google.com/github/P3109/Public/blob/main/Value%20Tables/make-value-tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Value tables of 8-bit floating point formats

This code allows the reader to experiment with properties of FP8 formats, and is produced in conjunction with the work of the IEEE P3109 working group on
floating point formats for machine learning, although is *not* an official output of that group.

The public outputs of of the group are available at https://github.com/P3109/Public and the interim report is at [PDF](https://github.com/P3109/Public/blob/main/Shared%20Reports/P3109%20WG%20Interim%20report.pdf)


In [9]:
%pip install pandas jinja2 airium ipytest gfloat==0.2.1

Note: you may need to restart the kernel to use updated packages.


In [11]:
from enum import Enum
from dataclasses import dataclass
import numpy as np
import airium
import pandas
from IPython.display import HTML

import ipytest
ipytest.autoconfig()

# Throw on overflow
np.seterr(over='raise')

def df_left_align(df):
  """
  Left-align dataframe for display
  """
  return df.style.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

pandas.set_option("render.max_rows", 13)

## FormatInfo

A small dataclass holding format information.  
See https://gfloat.readthedocs.io/en/latest/#gfloat.FormatInfo


In [37]:
from gfloat import FloatClass, FormatInfo, FloatValue, decode_float 
from gfloat.formats import *

other_formats = [format_info_ocp_e5m2, format_info_ocp_e4m3]
p3109_formats = [format_info_p3109(i) for i in range(1,8)]
p3109_and_ocp_formats = p3109_formats + other_formats
sub32_formats = p3109_and_ocp_formats + [format_info_binary16, format_info_bfloat16, format_info_binary32]

df = pandas.DataFrame(sub32_formats).set_index('name')

display(df_left_align(df)) # .sort_values(by='precision'))
print('precision: Number of significand bits (including implicit leading bit encoded by exponent>0)')
print('has_nz: Has negative zero (following e.g. LLVM Float8E5M2FNUZ)')
print('all_bits_one_full: An all-bits-one exponent corresponds to 2**(p-1) codes.',
      'If all of those codes are used for specials, e.g. Inf/NaN, this field is True.')


,k,precision,emax,has_nz,has_infs,num_high_nans,has_subnormals,is_signed,is_twos_complement
name,,,,,,,,,
p3109_p1,8,1,63,False,True,0,True,True,False
p3109_p2,8,2,31,False,True,0,True,True,False
p3109_p3,8,3,15,False,True,0,True,True,False
p3109_p4,8,4,7,False,True,0,True,True,False
p3109_p5,8,5,3,False,True,0,True,True,False
p3109_p6,8,6,1,False,True,0,True,True,False
p3109_p7,8,7,0,False,True,0,True,True,False
format_info_ocp_e5m2,8,3,15,True,True,3,True,True,False
format_info_ocp_e4m3,8,4,8,True,False,1,True,True,False


precision: Number of significand bits (including implicit leading bit encoded by exponent>0)
has_nz: Has negative zero (following e.g. LLVM Float8E5M2FNUZ)
all_bits_one_full: An all-bits-one exponent corresponds to 2**(p-1) codes. If all of those codes are used for specials, e.g. Inf/NaN, this field is True.


## FloatValue

A floating-point value in detail
See https://gfloat.readthedocs.io/en/latest/#gfloat.FloatValue


In [38]:

for fi in (format_info_p3109(precision=3), format_info_ocp_e5m2, format_info_ocp_e4m3):
  print('>>',fi)
  for ival in (0x00, 0x01, 0x40, 0x80, 0x7c, 0x7e, 0x7f):
    print(decode_float(fi, ival), sep='\n')

# Spot-check p3
fi = format_info_p3109(3)
dec = lambda ival: decode_float(fi, ival).fval

fclass = lambda ival: decode_float(fi, ival).fclass
assert dec(0x01) == 2.0 ** -17
assert dec(0x40) == 1.0
assert np.isnan(dec(0x80))
assert dec(0xff) == -np.inf
assert np.floor(np.log2(dec(0x7e))) == fi.emax

# Spot-check e5m2
fi = format_info_ocp_e5m2
assert dec(0x01) == 2.0 ** -16
assert dec(0x40) == 2.0
assert dec(0x80) == 0.0 and np.signbit(dec(0x80))
assert dec(0x7b) == 57344.0
assert dec(0x7c) == np.inf
assert np.floor(np.log2(dec(0x7b))) == fi.emax
assert dec(0xfc) == -np.inf
assert np.isnan(dec(0x7f))
assert fclass(0x80) == FloatClass.ZERO
assert fclass(0x00) == FloatClass.ZERO

# Spot-check e4m3
fi = format_info_ocp_e4m3
assert dec(0x40) == 2.0
assert dec(0x01) == 2.0 ** -9
assert dec(0x80) == 0.0 and np.signbit(dec(0x80))
assert np.isnan(dec(0x7f))
assert dec(0x7e) == 448.0
assert np.floor(np.log2(dec(0x7e))) == fi.emax

# Spot-check binary16
fi = format_info_binary16
assert dec(0x3c00) == 1.0
assert dec(0x3c01) == 1.0 + 2**-10
assert dec(0x4000) == 2.0
assert dec(0x0001) == 2**-24
assert dec(0x7bff) == 65504.0
assert np.isinf(dec(0x7c00))
assert np.isnan(dec(0x7c01))
assert np.isnan(dec(0x7fff))

# Spot-check bfloat16
fi = format_info_bfloat16
assert dec(0x3f80) == 1
assert dec(0x4000) == 2
assert dec(0x0001) == 2**-133
assert dec(0x4780) == 65536.0
assert np.isinf(dec(0x7f80))
assert np.isnan(dec(0x7f81))
assert np.isnan(dec(0x7fff))


>> p3109_p3
FloatValue(ival=0, fval=0.0, exp=0, expval=-15, significand=0, fsignificand=0.0, signbit=0, fclass=<FloatClass.ZERO: 3>)
FloatValue(ival=1, fval=7.62939453125e-06, exp=0, expval=-15, significand=1, fsignificand=0.25, signbit=0, fclass=<FloatClass.SUBNORMAL: 2>)
FloatValue(ival=64, fval=1.0, exp=16, expval=0, significand=0, fsignificand=1.0, signbit=0, fclass=<FloatClass.NORMAL: 1>)
FloatValue(ival=128, fval=nan, exp=0, expval=-15, significand=0, fsignificand=0.0, signbit=1, fclass=<FloatClass.NAN: 5>)
FloatValue(ival=124, fval=32768.0, exp=31, expval=15, significand=0, fsignificand=1.0, signbit=0, fclass=<FloatClass.NORMAL: 1>)
FloatValue(ival=126, fval=49152.0, exp=31, expval=15, significand=2, fsignificand=1.5, signbit=0, fclass=<FloatClass.NORMAL: 1>)
FloatValue(ival=127, fval=inf, exp=31, expval=15, significand=3, fsignificand=1.75, signbit=0, fclass=<FloatClass.INFINITE: 4>)
>> format_info_ocp_e5m2
FloatValue(ival=0, fval=0.0, exp=0, expval=-14, significand=0, fsignifi

#### Check subnormals

In [39]:
# Spot-check p4 with/without subnormals
fi = format_info_p3109(4)
dec = lambda ival: decode_float(fi, ival).fval
fclass = lambda ival: decode_float(fi, ival).fclass
minFloat = dec(0x01)
assert minFloat == 2.0 ** -10
assert dec(0x08) == 2.0 ** -7
assert fclass(0x07) == FloatClass.SUBNORMAL
assert fclass(0x08) == FloatClass.NORMAL

# Spot-check p4 with/without subnormals
fi.has_subnormals = False
minFloat_no_subnormals = dec(0x01)
assert minFloat_no_subnormals == 2.0 ** -8 + 2.0 ** -11
assert dec(0x08) == 2.0 ** -7
assert fclass(0x07) == FloatClass.NORMAL
assert fclass(0x08) == FloatClass.NORMAL

print(f'{minFloat=:.4f}')
print(f'{minFloat_no_subnormals=:.4f}')

print(f'Dynamic range increase with subnormals: x{minFloat_no_subnormals/minFloat}')


minFloat=0.0010
minFloat_no_subnormals=0.0044
Dynamic range increase with subnormals: x4.5


## Enumerate all values for a given format

In [40]:
# Get the FormatInfo
fi = format_info_p3109(precision=3)
print(fi)

# Generate values as a list of FloatValues
values = [decode_float(fi, i) for i in range(256)]

# Convert to dataframe (and hide the FormatInfo)
df_left_align(pandas.DataFrame(values)).hide().hide(subset=[*range(9,120), *range(131,251)])

with pandas.option_context("render.max_rows", 9):
  df = pandas.DataFrame(values)
  display(df_left_align(df[0:8]).hide())
  display(df_left_align(df[124:134]).hide())
  display(df_left_align(df[249:]).hide())


p3109_p3


ival,fval,exp,expval,significand,fsignificand,signbit,fclass
0,0.000000,0,-15,0,0.000000,0,FloatClass.ZERO
1,0.000008,0,-15,1,0.250000,0,FloatClass.SUBNORMAL
2,0.000015,0,-15,2,0.500000,0,FloatClass.SUBNORMAL
3,0.000023,0,-15,3,0.750000,0,FloatClass.SUBNORMAL
4,0.000031,1,-15,0,1.000000,0,FloatClass.NORMAL
5,0.000038,1,-15,1,1.250000,0,FloatClass.NORMAL
6,0.000046,1,-15,2,1.500000,0,FloatClass.NORMAL
7,0.000053,1,-15,3,1.750000,0,FloatClass.NORMAL


ival,fval,exp,expval,significand,fsignificand,signbit,fclass
124,32768.000000,31,15,0,1.000000,0,FloatClass.NORMAL
125,40960.000000,31,15,1,1.250000,0,FloatClass.NORMAL
126,49152.000000,31,15,2,1.500000,0,FloatClass.NORMAL
127,inf,31,15,3,1.750000,0,FloatClass.INFINITE
128,nan,0,-15,0,0.000000,1,FloatClass.NAN
129,-0.000008,0,-15,1,0.250000,1,FloatClass.SUBNORMAL
130,-0.000015,0,-15,2,0.500000,1,FloatClass.SUBNORMAL
131,-0.000023,0,-15,3,0.750000,1,FloatClass.SUBNORMAL
132,-0.000031,1,-15,0,1.000000,1,FloatClass.NORMAL
...,...,...,...,...,...,...,...


ival,fval,exp,expval,significand,fsignificand,signbit,fclass
249,-20480.000000,30,14,1,1.250000,1,FloatClass.NORMAL
250,-24576.000000,30,14,2,1.500000,1,FloatClass.NORMAL
251,-28672.000000,30,14,3,1.750000,1,FloatClass.NORMAL
252,-32768.000000,31,15,0,1.000000,1,FloatClass.NORMAL
253,-40960.000000,31,15,1,1.250000,1,FloatClass.NORMAL
254,-49152.000000,31,15,2,1.500000,1,FloatClass.NORMAL
255,-inf,31,15,3,1.750000,1,FloatClass.INFINITE


## Printing

String formatting for binary16 and F8 values

In [41]:
import struct
def b16_str(val) -> tuple[str,str]:
  """
  Represent VAL in binary16.

  If val does not convert exactly to binary16,
  returns "<Not16:{val}>"
  """
  with np.errstate(over="ignore"):
    b16 = np.float16(val)

  if float(b16) != val and np.isfinite(b16):
    # Finite, but not representable in float16
    return f'<Not16:{val}>',''
  b16_int = struct.unpack('!H',struct.pack('!e',b16))[0]

  # bitstr is of the form 0_00000_1100000000
  s = f'{b16_int:016b}'
  e_str = s[1:6]
  m_str = s[6:]
  bitstr = f'{s[0]}_{e_str}_{m_str}'

  # pow2str is of the form '+0b0.1100000000*2^-15', or '' for nonfinite values
  e = int(e_str,2) - 15
  m = int(m_str,2)
  leading_bit = 0 if e == -15 else 1
  signstr= '-' if s[0] == '1' else '+'
  if np.isfinite(b16):
    pow2str = f'{signstr}0b{leading_bit}.{m:010b}*2^{e}'
  else:
    pow2str = ''
  return bitstr,pow2str

for v in [3*2**-14, 3*2**-15, 3*2**-16, 3*2**-18]:
  print(b16_str(v))
print(b16_str(-np.inf))
print(b16_str(2 ** 16))
assert b16_str(3*2**-16) == ('0_00000_1100000000', '+0b0.1100000000*2^-15')

('0_00010_1000000000', '+0b1.1000000000*2^-13')
('0_00001_1000000000', '+0b1.1000000000*2^-14')
('0_00000_1100000000', '+0b0.1100000000*2^-15')
('0_00000_0011000000', '+0b0.0011000000*2^-15')
('1_11111_0000000000', '')
('0_11111_0000000000', '')


#### Render with underscores separating s_e_m

E.g `0_1011_110`.  For formats with zero significand bits or zero exponent bits, we use `0_1011110_` or `0__10111110`.

In [42]:
def str_bits_with_underscores(fi, fv):
  # 0_1011110_
  if fi.tSignificandBits == 0:
    return f'{fv.signbit}_{fv.exp:0{fi.expBits}b}_'

  # 0__1011110
  if fi.expBits == 0:
    return f'{fv.signbit}__{fv.significand:0{fi.tSignificandBits}b}'

  # 0_101_1110
  return f'{fv.signbit}_{fv.exp:0{fi.expBits}b}_{fv.significand:0{fi.tSignificandBits}b}'

fi = format_info_p3109(3)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == '0_10000_01'

fi = format_info_p3109(1)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == '0_1000001_'

fi = format_info_p3109(7)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == '0_1_000001'


## Collect value statistics for formats

Stats collected are:
 - lt1: Number of values with `0 < x < 1`
 - gt1: Number of values with `1<=x<Inf`
 - rt16: Round-trips to IEEE binary16
 - rt32: Round-trips to IEEE binary32 (True for all current formats, would not be true if emax(8) were 2**(-1/2).
 - maxNormal: Largest finite normal value, NaN if all finite values are subnormal
 - minSubnormal: Smallest positive subnormal value, NaN if no finite values are subnormal


In [43]:

def collect_stats(fi : FormatInfo):
  # Generate all values
  values = [decode_float(fi, i) for i in range(256)]
  df = pandas.DataFrame(values)
  
  # Extract format information parameters
  E=fi.expBits
  S=fi.tSignificandBits

  # Compute statistics: lt1,ge1
  fval = df['fval']
  total_01 = fval.between(0, 1, inclusive='neither').sum()
  total_1Inf = fval.between(1, np.inf, inclusive='left').sum()

  # Compute statistics: maxFinite,minFinite
  finite_vals = fval[np.isfinite(fval)]
  maxFinite = finite_vals.loc[finite_vals.idxmax()]
  minFinite = finite_vals.loc[finite_vals.idxmin()]

  # Compute statistics: maxNormal,minNormal
  normal_vals = fval[(df['fclass'] == FloatClass.NORMAL) & (fval>0)]
  maxNormal = normal_vals.loc[normal_vals.idxmax()] if normal_vals.any() else np.nan
  minNormal = normal_vals.loc[normal_vals.idxmin()] if normal_vals.any() else np.nan

  # Compute statistics: minSubnormal
  pos_subnormal = fval[(df['fclass'] == FloatClass.SUBNORMAL) & (fval>0)]
  maxSubnormal = pos_subnormal.loc[pos_subnormal.idxmax()] if pos_subnormal.any() else np.nan
  minSubnormal = pos_subnormal.loc[pos_subnormal.idxmin()] if pos_subnormal.any() else np.nan

  # Compute roundtrips: rt16, rt32
  with np.errstate(over='ignore'):
    rt16 = (np.float64(np.float16(fval)) == np.float64(fval)) | ~np.isfinite(fval)
    rt32 = (np.float64(np.float32(fval)) == np.float64(fval)) | ~np.isfinite(fval)

  rt16 = rt16.all()
  rt32 = rt32.all()
  assert rt32 # If not, we should include rt32 in the table
  
  # Assemble tuple
  return dict(name=fi.name,P=fi.precision,E=E,T=S,
            lt1=total_01,ge1=total_1Inf,
            rt16=rt16,
            maxFinite=maxFinite,minFinite=minFinite,
            maxNormal=maxNormal,minNormal=minNormal,
            minSubnormal=minSubnormal,maxSubnormal=maxSubnormal)

legends = {
  'name': 'Format',
  'P': 'Precision',
  'E': 'Exponent field width in bits',
  'T': 'Trailing significand field width in bits',
  'lt1':  'Number of values <code>x</code> such that <code>0 < x < 1</code>',
  'ge1':  'Number of values <code>x</code> such that <code>1 <= x < Inf</code>',
  'rt16':  'Round-trips to IEEE binary16',
  'maxFinite':  'Largest finite value',
  'minFinite':  'Smallest finite value',
  'maxNormal':  'Largest finite normal value, NaN if all finite values are subnormal',
  'minNormal':  'Smallest positive normal value, NaN if all finite values are subnormal',
  'minSubnormal':  'Smallest positive subnormal value, NaN if no finite values are subnormal',
  'maxSubnormal':  'Largest subnormal value, NaN if no finite values are subnormal'
}

stats = [collect_stats(fi) for fi in p3109_and_ocp_formats]
df = pandas.DataFrame(stats)
HTML("""<h3> Summary statistics for formats </h3>
<ul>""" + 
"\n".join(f'<li> {n}: {legends[n]}' for n in df.columns) + """
</ul>
""" + df_left_align(df).hide().to_html())

name,P,E,T,lt1,ge1,rt16,maxFinite,minFinite,maxNormal,minNormal,minSubnormal,maxSubnormal
p3109_p1,1,7,0,62,64,False,9223372036854775808.000000,-9223372036854775808.000000,9223372036854775808.000000,0.000000,nan,nan
p3109_p2,2,6,1,63,63,False,2147483648.000000,-2147483648.000000,2147483648.000000,0.000000,0.000000,0.000000
p3109_p3,3,5,2,63,63,True,49152.000000,-49152.000000,49152.000000,0.000031,0.000008,0.000023
p3109_p4,4,4,3,63,63,True,224.000000,-224.000000,224.000000,0.007812,0.000977,0.006836
p3109_p5,5,3,4,63,63,True,15.000000,-15.000000,15.000000,0.125000,0.007812,0.117188
p3109_p6,6,2,5,63,63,True,3.875000,-3.875000,3.875000,0.500000,0.015625,0.484375
p3109_p7,7,1,6,63,63,True,1.968750,-1.968750,1.968750,1.000000,0.015625,0.984375
format_info_ocp_e5m2,3,5,2,59,64,True,57344.000000,-57344.000000,57344.000000,0.000061,0.000015,0.000046
format_info_ocp_e4m3,4,4,3,55,71,True,448.000000,-448.000000,448.000000,0.015625,0.001953,0.013672


## Render values as exact fractions * 2^e


In [44]:
from gfloat.printing import float_pow2str

df_pow2 = df.copy()
for field in ("maxFinite","minFinite","maxNormal","minNormal","minSubnormal","maxSubnormal"):
  df_pow2[field] = df_pow2[field].map(float_pow2str)
df_left_align(df_pow2).hide()

name,P,E,T,lt1,ge1,rt16,maxFinite,minFinite,maxNormal,minNormal,minSubnormal,maxSubnormal
p3109_p1,1,7,0,62,64,False,1*2^63,-1*2^63,1*2^63,1*2^-62,nan,nan
p3109_p2,2,6,1,63,63,False,1*2^31,-1*2^31,1*2^31,1*2^-31,1*2^-32,1*2^-32
p3109_p3,3,5,2,63,63,True,3/2*2^15,-3/2*2^15,3/2*2^15,1*2^-15,1*2^-17,3/2*2^-16
p3109_p4,4,4,3,63,63,True,7/4*2^7,-7/4*2^7,7/4*2^7,1*2^-7,1*2^-10,7/4*2^-8
p3109_p5,5,3,4,63,63,True,15/8*2^3,-15/8*2^3,15/8*2^3,1*2^-3,1*2^-7,15/8*2^-4
p3109_p6,6,2,5,63,63,True,31/16*2^1,-31/16*2^1,31/16*2^1,1*2^-1,1*2^-6,31/16*2^-2
p3109_p7,7,1,6,63,63,True,63/32*2^0,-63/32*2^0,63/32*2^0,1*2^0,1*2^-6,63/32*2^-1
format_info_ocp_e5m2,3,5,2,59,64,True,7/4*2^15,-7/4*2^15,7/4*2^15,1*2^-14,1*2^-16,3/2*2^-15
format_info_ocp_e4m3,4,4,3,55,71,True,7/4*2^8,-7/4*2^8,7/4*2^8,1*2^-6,1*2^-9,7/4*2^-7


## Generate LaTeX table of min/max values

In [45]:
df_pow2[["minSubnormal","maxSubnormal","minNormal","maxNormal","maxFinite"]]


,minSubnormal,maxSubnormal,minNormal,maxNormal,maxFinite
0,nan,nan,1*2^-62,1*2^63,1*2^63
1,1*2^-32,1*2^-32,1*2^-31,1*2^31,1*2^31
2,1*2^-17,3/2*2^-16,1*2^-15,3/2*2^15,3/2*2^15
3,1*2^-10,7/4*2^-8,1*2^-7,7/4*2^7,7/4*2^7
4,1*2^-7,15/8*2^-4,1*2^-3,15/8*2^3,15/8*2^3
5,1*2^-6,31/16*2^-2,1*2^-1,31/16*2^1,31/16*2^1
6,1*2^-6,63/32*2^-1,1*2^0,63/32*2^0,63/32*2^0
7,1*2^-16,3/2*2^-15,1*2^-14,7/4*2^15,7/4*2^15
8,1*2^-9,7/4*2^-7,1*2^-6,7/4*2^8,7/4*2^8


In [46]:
import pathlib
dir = pathlib.Path("latex")
dir.mkdir(parents=True, exist_ok=True)

tbl3 = df_pow2[["name", "minSubnormal","maxSubnormal","minNormal","maxNormal","maxFinite"]]
header=["Format", "minSubnormal","maxSubnormal","minNormal","maxNormal","maxFinite"]
latex = tbl3.to_latex(header=header, index=False, float_format=float_pow2str, na_rep='N/A')

import re 
latex = re.sub('([\d./]+)\*2.textasciicircum ([\d-]+)','$\\\\binaryfloat{\\1}{\\2}$', latex)
latex = latex.replace('p3109_p', 'p')
latex = latex.replace('ocp_', 'ocp\_')

with open('latex/tbl-extremalvalues.tex', 'w') as f:
  pr = lambda x: print(x,file=f)
  pr('% File: tbl-extremalvalues.tex')
  pr('% Generated from https://github.com/P3109/Public/blob/main/Value%20Tables/make-value-tables.ipynb')
  pr(latex)

!grep -H . latex/tbl-extremalvalues.tex

latex/tbl-extremalvalues.tex:% File: tbl-extremalvalues.tex
latex/tbl-extremalvalues.tex:% Generated from https://github.com/P3109/Public/blob/main/Value%20Tables/make-value-tables.ipynb
latex/tbl-extremalvalues.tex:\begin{tabular}{llllll}
latex/tbl-extremalvalues.tex:\toprule
latex/tbl-extremalvalues.tex:Format & minSubnormal & maxSubnormal & minNormal & maxNormal & maxFinite \\
latex/tbl-extremalvalues.tex:\midrule
latex/tbl-extremalvalues.tex:p1 & nan & nan & 1*2^-62 & 1*2^63 & 1*2^63 \\
latex/tbl-extremalvalues.tex:p2 & 1*2^-32 & 1*2^-32 & 1*2^-31 & 1*2^31 & 1*2^31 \\
latex/tbl-extremalvalues.tex:p3 & 1*2^-17 & 3/2*2^-16 & 1*2^-15 & 3/2*2^15 & 3/2*2^15 \\
latex/tbl-extremalvalues.tex:p4 & 1*2^-10 & 7/4*2^-8 & 1*2^-7 & 7/4*2^7 & 7/4*2^7 \\
latex/tbl-extremalvalues.tex:p5 & 1*2^-7 & 15/8*2^-4 & 1*2^-3 & 15/8*2^3 & 15/8*2^3 \\
latex/tbl-extremalvalues.tex:p6 & 1*2^-6 & 31/16*2^-2 & 1*2^-1 & 31/16*2^1 & 31/16*2^1 \\
latex/tbl-extremalvalues.tex:p7 & 1*2^-6 & 63/32*2^-1 & 1*2^0 & 63/32*

## Value Tables

Print one-line summaries of each float value in the form
```text
Code Binary     = Exact binary E =  Float16 equivalent Float16 binary E    = Float Value
0x21 0_0100_001 = +0b1.001*2^-4  = 0_01011_0010000000 +0b1.0010000000*2^-4 = ~0.0703
```

In [49]:
from gfloat import float_tilde_unless_roundtrip_str

def str_tablerow(fv : FloatValue, show_b16_info=True, vs_width=14, vs_d=8):
  """
  Create a string of the form
    0x41 0_10000_01 = +0b1.01*2^0   = 1.25
  optionally adding binary16 info
    0x41 0_10000_01 = +0b1.01*2^0   = 0_01111_0100000000 +0b1.0100000000*2^0 = 1.25
  """
  text = []

  # 0x45 0_1000_101
  text.append(f'0x{fv.ival:02x} {str_bits_with_underscores(fi,fv)}')
  
  finite_nonzero = np.isfinite(fv.fval) and fv.fval != 0
  
  #  = +0b1.101*2^-7 =
  if finite_nonzero:
    b = '0' if fv.fclass == FloatClass.SUBNORMAL else '1'
    signstr = "-" if fv.signbit else "+"
    binary_pow2 = f'{signstr}0b{b}.{fv.significand:0{fi.tSignificandBits}b}*2^{fv.expval:<3}'
    text.append(binary_pow2)

  if show_b16_info and finite_nonzero:
    b16_binary_str,b16_bscistr = b16_str(fv.fval)
    text.append(f'{b16_binary_str} {b16_bscistr}')

  # 1.125
  text.append(float_tilde_unless_roundtrip_str(fv.fval, width=vs_width, d=vs_d))

  # Return tuple
  return " = ".join(text)

for fi in (format_info_p3109(4), format_info_p3109(1)):
  print(fi.name)
  for i in (0x00, 0x01, 0x07, 0x21, 0x40, 0x41, 0x7e, 0x7f, 0x80, 0x81, 0xe6, 0xfe,0xff):
    print(str_tablerow(decode_float(fi, i), show_b16_info=True, vs_width=8, vs_d=4))


p3109_p4
0x00 0_0000_000 = 0.0
0x01 0_0000_001 = +0b0.001*2^-7  = 0_00101_0000000000 +0b1.0000000000*2^-10 = ~0.0010
0x07 0_0000_111 = +0b0.111*2^-7  = 0_00111_1100000000 +0b1.1100000000*2^-8 = ~0.0068
0x21 0_0100_001 = +0b1.001*2^-4  = 0_01011_0010000000 +0b1.0010000000*2^-4 = ~0.0703
0x40 0_1000_000 = +0b1.000*2^0   = 0_01111_0000000000 +0b1.0000000000*2^0 = 1.0
0x41 0_1000_001 = +0b1.001*2^0   = 0_01111_0010000000 +0b1.0010000000*2^0 = 1.125
0x7e 0_1111_110 = +0b1.110*2^7   = 0_10110_1100000000 +0b1.1100000000*2^7 = 224.0
0x7f 0_1111_111 = inf
0x80 1_0000_000 = nan
0x81 1_0000_001 = -0b0.001*2^-7  = 1_00101_0000000000 -0b1.0000000000*2^-10 = ~-0.0010
0xe6 1_1100_110 = -0b1.110*2^4   = 1_10011_1100000000 -0b1.1100000000*2^4 = -28.0
0xfe 1_1111_110 = -0b1.110*2^7   = 1_10110_1100000000 -0b1.1100000000*2^7 = -224.0
0xff 1_1111_111 = -inf
p3109_p1
0x00 0_0000000_ = 0.0
0x01 0_0000001_ = +0b1.0*2^-62 = <Not16:2.168404344971009e-19>  = ~2.168e-19
0x07 0_0000111_ = +0b1.0*2^-56 = <Not16:1.

In [50]:
def table_style(fv):
  if fv.fclass == FloatClass.SUBNORMAL:
     return "subnormal"
  
  if fv.fclass == FloatClass.NORMAL:
     return "normal"

  if fv.fclass == FloatClass.ZERO and not fv.signbit:
     return "normal"

  # Everyting else is special
  return "special"

assert table_style(decode_float(format_info_ocp_e5m2, 0x80)) == "special"
assert table_style(decode_float(format_info_ocp_e5m2, 0x01)) == "subnormal"
assert table_style(decode_float(format_info_ocp_e5m2, 0x7f)) == "special"


def mktbl(fi : FormatInfo, file):
  # Make tables
  cols = 4
  rows = 256//cols

  for i in range(0,rows):
    out_row = []
    for n in range(i,256,rows):
      fv = decode_float(fi, n)
      text = str_tablerow(fv, show_b16_info=False)
      style = table_style(fv)

      # 2^-7  -> \pow{-7}
      text = re.sub(r'\*2\^([-0-9]+)', r'\\pow{\1}', text)
      # 1.234E7  -> \e{1.234}{7}
      text = re.sub(r'([0-9.+-]+)[eE]([0-9]+)', r'\\e{\1}{\2}', text)
      # 1.234E-7  -> \e{1.234}{-7}
      text = re.sub(r'([0-9.+-]+)[eE]-([0-9]+)', r'\\e{\1}{\\neg{\2}}', text)
      # -1.234 EOL  -> \f{-1.234}
      text = re.sub(r'([0-9.+-]+)$', r'\\f{\1}', text)
      #\f{-1.234} -> \f{\neg{1.234}}
      text = re.sub(r'\\(f|pow)\{-([0-9.]+)\}', r'\\\1{\\neg{\2}}', text)
      
      # ^0x44  -> 0x44~
      text = re.sub('^(0x[0-9a-f]+) ', '\\1 = ', text)
      text = text.replace('_', '\_')
      text = text.replace('= ~', '\\approx ')
      out_row += [f'\\{style}{{{text}}}']
    print(*out_row, sep="&\n", end=r'\\'+"\n", file=file)

for fi in p3109_and_ocp_formats:
  filename = f"value-table-{fi.name}.tex"
  print(f'Saving to {dir / filename}')
  with open(dir / filename, "w") as f:
    print('% Autogenerated from make-value-tables.ipynb', file=f)
    mktbl(fi, f)


Saving to latex/value-table-p3109_p1.tex
Saving to latex/value-table-p3109_p2.tex
Saving to latex/value-table-p3109_p3.tex
Saving to latex/value-table-p3109_p4.tex
Saving to latex/value-table-p3109_p5.tex
Saving to latex/value-table-p3109_p6.tex
Saving to latex/value-table-p3109_p7.tex
Saving to latex/value-table-format_info_ocp_e5m2.tex
Saving to latex/value-table-format_info_ocp_e4m3.tex


## Generate HTML Value Tables

In [51]:
def mktbl(fi : FormatInfo, short=False, **kw):
  # Make tables
  cols = 4
  rows = 256//cols

  style = f'''
  body {{
  }}
  table {{
    width:100%;
    margin: 0pt;
    font-family: monospace;
    font-size: small;
    font-weight: bold;
    border-collapse: collapse;
  }}

  tr.blankrow {{
    height: 4ex;
    vertical-align: top;
  }}
  
  td {{
    text-align: left;
    border: solid 2px #ccc;
    width: {98/cols}%;
  }}
  
  .special {{
    color: #874723;
  }}
  
  .subnormal {{
    color: #0121a7;
  }}
  
  .normal {{
  }}
  
  pre {{
    margin: 1pt 1pt 1pt 13pt;
    display: inline;
  }}
'''

  title = f"FP8 Value Table, {fi.name}"
  a = airium.Airium()
  a('<!DOCTYPE html>')
  with a.html():
    with a.head():
        # a.meta('http-equiv="refresh" content="1"') # for rapid testing
        a.meta(charset="utf-8")
        a.title(_t=title)
        a.style(_t=style)

    with a.body():
        a.h3(_t=title)

        with a.table():
          for i in range(0,rows):
            if short and (16 <= i < 60):
              if i == 16:
                a.tr(klass='blankrow').td('...')
              continue
            trklass='blankrow' if i > 0 and i%16 == 0 else ''
            with a.tr(klass=trklass):
              for n in range(i,256,rows):
                fv = decode_float(fi, n)
                text = str_tablerow(fv, show_b16_info=False, **kw)
                a.td(klass=table_style(fv)).pre(_t=text)

  return str(a)

HTML(mktbl(format_info_p3109(4), True, vs_width=8, vs_d=3))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^0 = 1.0,0x80 1_0000_000 = nan,0xc0 1_1000_000 = -0b1.000*2^0 = -1.0
0x01 0_0000_001 = +0b0.001*2^-7 = ~0.001,0x41 0_1000_001 = +0b1.001*2^0 = 1.125,0x81 1_0000_001 = -0b0.001*2^-7 = ~-0.001,0xc1 1_1000_001 = -0b1.001*2^0 = -1.125
0x02 0_0000_010 = +0b0.010*2^-7 = ~0.002,0x42 0_1000_010 = +0b1.010*2^0 = 1.25,0x82 1_0000_010 = -0b0.010*2^-7 = ~-0.002,0xc2 1_1000_010 = -0b1.010*2^0 = -1.25
0x03 0_0000_011 = +0b0.011*2^-7 = ~0.003,0x43 0_1000_011 = +0b1.011*2^0 = 1.375,0x83 1_0000_011 = -0b0.011*2^-7 = ~-0.003,0xc3 1_1000_011 = -0b1.011*2^0 = -1.375
0x04 0_0000_100 = +0b0.100*2^-7 = ~0.004,0x44 0_1000_100 = +0b1.100*2^0 = 1.5,0x84 1_0000_100 = -0b0.100*2^-7 = ~-0.004,0xc4 1_1000_100 = -0b1.100*2^0 = -1.5
0x05 0_0000_101 = +0b0.101*2^-7 = ~0.005,0x45 0_1000_101 = +0b1.101*2^0 = 1.625,0x85 1_0000_101 = -0b0.101*2^-7 = ~-0.005,0xc5 1_1000_101 = -0b1.101*2^0 = -1.625
0x06 0_0000_110 = +0b0.110*2^-7 = ~0.006,0x46 0_1000_110 = +0b1.110*2^0 = 1.75,0x86 1_0000_110 = -0b0.110*2^-7 = ~-0.006,0xc6 1_1000_110 = -0b1.110*2^0 = -1.75
0x07 0_0000_111 = +0b0.111*2^-7 = ~0.007,0x47 0_1000_111 = +0b1.111*2^0 = 1.875,0x87 1_0000_111 = -0b0.111*2^-7 = ~-0.007,0xc7 1_1000_111 = -0b1.111*2^0 = -1.875
0x08 0_0001_000 = +0b1.000*2^-7 = ~0.008,0x48 0_1001_000 = +0b1.000*2^1 = 2.0,0x88 1_0001_000 = -0b1.000*2^-7 = ~-0.008,0xc8 1_1001_000 = -0b1.000*2^1 = -2.0
0x09 0_0001_001 = +0b1.001*2^-7 = ~0.009,0x49 0_1001_001 = +0b1.001*2^1 = 2.25,0x89 1_0001_001 = -0b1.001*2^-7 = ~-0.009,0xc9 1_1001_001 = -0b1.001*2^1 = -2.25
0x0a 0_0001_010 = +0b1.010*2^-7 = ~0.010,0x4a 0_1001_010 = +0b1.010*2^1 = 2.5,0x8a 1_0001_010 = -0b1.010*2^-7 = ~-0.010,0xca 1_1001_010 = -0b1.010*2^1 = -2.5


In [52]:
# What would the table look like without subnormals
fi = format_info_p3109(4)
fi.name += ' [without subnormals]'
fi.has_subnormals = False
HTML(mktbl(fi, short=True, vs_width=8, vs_d=3))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^0 = 1.0,0x80 1_0000_000 = nan,0xc0 1_1000_000 = -0b1.000*2^0 = -1.0
0x01 0_0000_001 = +0b1.001*2^-8 = ~0.004,0x41 0_1000_001 = +0b1.001*2^0 = 1.125,0x81 1_0000_001 = -0b1.001*2^-8 = ~-0.004,0xc1 1_1000_001 = -0b1.001*2^0 = -1.125
0x02 0_0000_010 = +0b1.010*2^-8 = ~0.005,0x42 0_1000_010 = +0b1.010*2^0 = 1.25,0x82 1_0000_010 = -0b1.010*2^-8 = ~-0.005,0xc2 1_1000_010 = -0b1.010*2^0 = -1.25
0x03 0_0000_011 = +0b1.011*2^-8 = ~0.005,0x43 0_1000_011 = +0b1.011*2^0 = 1.375,0x83 1_0000_011 = -0b1.011*2^-8 = ~-0.005,0xc3 1_1000_011 = -0b1.011*2^0 = -1.375
0x04 0_0000_100 = +0b1.100*2^-8 = ~0.006,0x44 0_1000_100 = +0b1.100*2^0 = 1.5,0x84 1_0000_100 = -0b1.100*2^-8 = ~-0.006,0xc4 1_1000_100 = -0b1.100*2^0 = -1.5
0x05 0_0000_101 = +0b1.101*2^-8 = ~0.006,0x45 0_1000_101 = +0b1.101*2^0 = 1.625,0x85 1_0000_101 = -0b1.101*2^-8 = ~-0.006,0xc5 1_1000_101 = -0b1.101*2^0 = -1.625
0x06 0_0000_110 = +0b1.110*2^-8 = ~0.007,0x46 0_1000_110 = +0b1.110*2^0 = 1.75,0x86 1_0000_110 = -0b1.110*2^-8 = ~-0.007,0xc6 1_1000_110 = -0b1.110*2^0 = -1.75
0x07 0_0000_111 = +0b1.111*2^-8 = ~0.007,0x47 0_1000_111 = +0b1.111*2^0 = 1.875,0x87 1_0000_111 = -0b1.111*2^-8 = ~-0.007,0xc7 1_1000_111 = -0b1.111*2^0 = -1.875
0x08 0_0001_000 = +0b1.000*2^-7 = ~0.008,0x48 0_1001_000 = +0b1.000*2^1 = 2.0,0x88 1_0001_000 = -0b1.000*2^-7 = ~-0.008,0xc8 1_1001_000 = -0b1.000*2^1 = -2.0
0x09 0_0001_001 = +0b1.001*2^-7 = ~0.009,0x49 0_1001_001 = +0b1.001*2^1 = 2.25,0x89 1_0001_001 = -0b1.001*2^-7 = ~-0.009,0xc9 1_1001_001 = -0b1.001*2^1 = -2.25
0x0a 0_0001_010 = +0b1.010*2^-7 = ~0.010,0x4a 0_1001_010 = +0b1.010*2^1 = 2.5,0x8a 1_0001_010 = -0b1.010*2^-7 = ~-0.010,0xca 1_1001_010 = -0b1.010*2^1 = -2.5


In [54]:
## Write HTML to disk
import pathlib
dir = pathlib.Path("html")
dir.mkdir(parents=True, exist_ok=True)

a = airium.Airium()
a('<!DOCTYPE html>')
a('<!-- Autogenerated from make-value-tables.ipynb -->')
a.head().title(_t='F8 value tables')
with a.body():
  a.h2(_t='F8 value tables')
  with a.ol():
    for fi in p3109_and_ocp_formats:
      html_str = mktbl(fi)
      filename = f"value-table-{fi.name}.html"
      print(f'Saving to {dir / filename}')
      a.li().a(href=filename).code(_t=fi.name)
      with open(dir / filename, "w") as f:
        f.write(html_str)

index_filename = dir / 'index.html'
print(f'Saving {index_filename}')
with open(index_filename, "w") as index:
  index.write(str(a))


Saving to html/value-table-p3109_p1.html
Saving to html/value-table-p3109_p2.html
Saving to html/value-table-p3109_p3.html
Saving to html/value-table-p3109_p4.html
Saving to html/value-table-p3109_p5.html
Saving to html/value-table-p3109_p6.html
Saving to html/value-table-p3109_p7.html
Saving to html/value-table-format_info_ocp_e5m2.html
Saving to html/value-table-format_info_ocp_e4m3.html
Saving html/index.html


## Show full table for p=4

In [55]:
HTML(mktbl(format_info_p3109(4), short=True))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^0 = 1.0,0x80 1_0000_000 = nan,0xc0 1_1000_000 = -0b1.000*2^0 = -1.0
0x01 0_0000_001 = +0b0.001*2^-7 = 0.0009765625,0x41 0_1000_001 = +0b1.001*2^0 = 1.125,0x81 1_0000_001 = -0b0.001*2^-7 = -0.0009765625,0xc1 1_1000_001 = -0b1.001*2^0 = -1.125
0x02 0_0000_010 = +0b0.010*2^-7 = 0.001953125,0x42 0_1000_010 = +0b1.010*2^0 = 1.25,0x82 1_0000_010 = -0b0.010*2^-7 = -0.001953125,0xc2 1_1000_010 = -0b1.010*2^0 = -1.25
0x03 0_0000_011 = +0b0.011*2^-7 = 0.0029296875,0x43 0_1000_011 = +0b1.011*2^0 = 1.375,0x83 1_0000_011 = -0b0.011*2^-7 = -0.0029296875,0xc3 1_1000_011 = -0b1.011*2^0 = -1.375
0x04 0_0000_100 = +0b0.100*2^-7 = 0.00390625,0x44 0_1000_100 = +0b1.100*2^0 = 1.5,0x84 1_0000_100 = -0b0.100*2^-7 = -0.00390625,0xc4 1_1000_100 = -0b1.100*2^0 = -1.5
0x05 0_0000_101 = +0b0.101*2^-7 = 0.0048828125,0x45 0_1000_101 = +0b1.101*2^0 = 1.625,0x85 1_0000_101 = -0b0.101*2^-7 = -0.0048828125,0xc5 1_1000_101 = -0b1.101*2^0 = -1.625
0x06 0_0000_110 = +0b0.110*2^-7 = 0.005859375,0x46 0_1000_110 = +0b1.110*2^0 = 1.75,0x86 1_0000_110 = -0b0.110*2^-7 = -0.005859375,0xc6 1_1000_110 = -0b1.110*2^0 = -1.75
0x07 0_0000_111 = +0b0.111*2^-7 = 0.0068359375,0x47 0_1000_111 = +0b1.111*2^0 = 1.875,0x87 1_0000_111 = -0b0.111*2^-7 = -0.0068359375,0xc7 1_1000_111 = -0b1.111*2^0 = -1.875
0x08 0_0001_000 = +0b1.000*2^-7 = 0.0078125,0x48 0_1001_000 = +0b1.000*2^1 = 2.0,0x88 1_0001_000 = -0b1.000*2^-7 = -0.0078125,0xc8 1_1001_000 = -0b1.000*2^1 = -2.0
0x09 0_0001_001 = +0b1.001*2^-7 = 0.0087890625,0x49 0_1001_001 = +0b1.001*2^1 = 2.25,0x89 1_0001_001 = -0b1.001*2^-7 = -0.0087890625,0xc9 1_1001_001 = -0b1.001*2^1 = -2.25
0x0a 0_0001_010 = +0b1.010*2^-7 = 0.009765625,0x4a 0_1001_010 = +0b1.010*2^1 = 2.5,0x8a 1_0001_010 = -0b1.010*2^-7 = -0.009765625,0xca 1_1001_010 = -0b1.010*2^1 = -2.5


## Format tables: OCP formats

In [56]:
HTML(mktbl(format_info_ocp_e5m2, short=True))

0x00 0_0000_000 = 0.0,0x40 0_10000_000 = +0b1.000*2^1 = 2.0,0x80 1_0000_000 = -0.0,0xc0 1_10000_000 = -0b1.000*2^1 = -2.0
0x01 0_0000_001 = +0b0.001*2^-14 = ~1.5258789e-05,0x41 0_10000_001 = +0b1.001*2^1 = 2.5,0x81 1_0000_001 = -0b0.001*2^-14 = ~-1.5258789e-05,0xc1 1_10000_001 = -0b1.001*2^1 = -2.5
0x02 0_0000_010 = +0b0.010*2^-14 = ~3.0517578e-05,0x42 0_10000_010 = +0b1.010*2^1 = 3.0,0x82 1_0000_010 = -0b0.010*2^-14 = ~-3.0517578e-05,0xc2 1_10000_010 = -0b1.010*2^1 = -3.0
0x03 0_0000_011 = +0b0.011*2^-14 = ~4.5776367e-05,0x43 0_10000_011 = +0b1.011*2^1 = 3.5,0x83 1_0000_011 = -0b0.011*2^-14 = ~-4.5776367e-05,0xc3 1_10000_011 = -0b1.011*2^1 = -3.5
0x04 0_0001_000 = +0b1.000*2^-14 = ~6.1035156e-05,0x44 0_10001_000 = +0b1.000*2^2 = 4.0,0x84 1_0001_000 = -0b1.000*2^-14 = ~-6.1035156e-05,0xc4 1_10001_000 = -0b1.000*2^2 = -4.0
0x05 0_0001_001 = +0b1.001*2^-14 = ~7.6293945e-05,0x45 0_10001_001 = +0b1.001*2^2 = 5.0,0x85 1_0001_001 = -0b1.001*2^-14 = ~-7.6293945e-05,0xc5 1_10001_001 = -0b1.001*2^2 = -5.0
0x06 0_0001_010 = +0b1.010*2^-14 = ~9.1552734e-05,0x46 0_10001_010 = +0b1.010*2^2 = 6.0,0x86 1_0001_010 = -0b1.010*2^-14 = ~-9.1552734e-05,0xc6 1_10001_010 = -0b1.010*2^2 = -6.0
0x07 0_0001_011 = +0b1.011*2^-14 = ~0.00010681,0x47 0_10001_011 = +0b1.011*2^2 = 7.0,0x87 1_0001_011 = -0b1.011*2^-14 = ~-0.00010681,0xc7 1_10001_011 = -0b1.011*2^2 = -7.0
0x08 0_0010_000 = +0b1.000*2^-13 = ~0.00012207,0x48 0_10010_000 = +0b1.000*2^3 = 8.0,0x88 1_0010_000 = -0b1.000*2^-13 = ~-0.00012207,0xc8 1_10010_000 = -0b1.000*2^3 = -8.0
0x09 0_0010_001 = +0b1.001*2^-13 = ~0.00015259,0x49 0_10010_001 = +0b1.001*2^3 = 10.0,0x89 1_0010_001 = -0b1.001*2^-13 = ~-0.00015259,0xc9 1_10010_001 = -0b1.001*2^3 = -10.0
0x0a 0_0010_010 = +0b1.010*2^-13 = ~0.00018311,0x4a 0_10010_010 = +0b1.010*2^3 = 12.0,0x8a 1_0010_010 = -0b1.010*2^-13 = ~-0.00018311,0xca 1_10010_010 = -0b1.010*2^3 = -12.0


In [57]:
HTML(mktbl(format_info_ocp_e4m3, short=True))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^1 = 2.0,0x80 1_0000_000 = -0.0,0xc0 1_1000_000 = -0b1.000*2^1 = -2.0
0x01 0_0000_001 = +0b0.001*2^-6 = 0.001953125,0x41 0_1000_001 = +0b1.001*2^1 = 2.25,0x81 1_0000_001 = -0b0.001*2^-6 = -0.001953125,0xc1 1_1000_001 = -0b1.001*2^1 = -2.25
0x02 0_0000_010 = +0b0.010*2^-6 = 0.00390625,0x42 0_1000_010 = +0b1.010*2^1 = 2.5,0x82 1_0000_010 = -0b0.010*2^-6 = -0.00390625,0xc2 1_1000_010 = -0b1.010*2^1 = -2.5
0x03 0_0000_011 = +0b0.011*2^-6 = 0.005859375,0x43 0_1000_011 = +0b1.011*2^1 = 2.75,0x83 1_0000_011 = -0b0.011*2^-6 = -0.005859375,0xc3 1_1000_011 = -0b1.011*2^1 = -2.75
0x04 0_0000_100 = +0b0.100*2^-6 = 0.0078125,0x44 0_1000_100 = +0b1.100*2^1 = 3.0,0x84 1_0000_100 = -0b0.100*2^-6 = -0.0078125,0xc4 1_1000_100 = -0b1.100*2^1 = -3.0
0x05 0_0000_101 = +0b0.101*2^-6 = 0.009765625,0x45 0_1000_101 = +0b1.101*2^1 = 3.25,0x85 1_0000_101 = -0b0.101*2^-6 = -0.009765625,0xc5 1_1000_101 = -0b1.101*2^1 = -3.25
0x06 0_0000_110 = +0b0.110*2^-6 = 0.01171875,0x46 0_1000_110 = +0b1.110*2^1 = 3.5,0x86 1_0000_110 = -0b0.110*2^-6 = -0.01171875,0xc6 1_1000_110 = -0b1.110*2^1 = -3.5
0x07 0_0000_111 = +0b0.111*2^-6 = 0.013671875,0x47 0_1000_111 = +0b1.111*2^1 = 3.75,0x87 1_0000_111 = -0b0.111*2^-6 = -0.013671875,0xc7 1_1000_111 = -0b1.111*2^1 = -3.75
0x08 0_0001_000 = +0b1.000*2^-6 = 0.015625,0x48 0_1001_000 = +0b1.000*2^2 = 4.0,0x88 1_0001_000 = -0b1.000*2^-6 = -0.015625,0xc8 1_1001_000 = -0b1.000*2^2 = -4.0
0x09 0_0001_001 = +0b1.001*2^-6 = 0.017578125,0x49 0_1001_001 = +0b1.001*2^2 = 4.5,0x89 1_0001_001 = -0b1.001*2^-6 = -0.017578125,0xc9 1_1001_001 = -0b1.001*2^2 = -4.5
0x0a 0_0001_010 = +0b1.010*2^-6 = 0.01953125,0x4a 0_1001_010 = +0b1.010*2^2 = 5.0,0x8a 1_0001_010 = -0b1.010*2^-6 = -0.01953125,0xca 1_1001_010 = -0b1.010*2^2 = -5.0
